# koGPT2를 이용한 챗봇 학습 & 테스트 실습

- koGPT2를 사전학습 모델로 두고, 별도 데이터를 이용해 추가 학습(fine tuning)을 한다.

In [1]:
import numpy as np
import pandas as pd
import torch

# !pip install pytorch_lightning==1.6.0 버전으로 받기
# ->  pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast,GPT2LMHeadModel
import re

In [2]:
from preprocess.clean_data import ChatbotDataset
from fine_tuning.train import collate_batch

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


# I. 데이터 불러오기(추가 학습용)

In [3]:
import urllib.request

# 실행시 현재 파일과 같은 경로에 ChatBotData.csv 파일 생성됨
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")

# Test 용으로 300개 데이터만 처리한다.
Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
num_workers = 2
train_dataloader = DataLoader(train_set, 
                              batch_size=32, 
                              num_workers=num_workers, 
                              shuffle=True, 
                              collate_fn=collate_batch,)

In [6]:
train_set._data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
295,궁금하지?,안 궁금해요.,0
296,궁금해,자세히 말씀해주세요.,0
297,궁금해 알려줘,자세히 말씀해주세요.,0
298,귀 아파,병원에 가세요.,0


# II. 전처리: ChatbotDataset 클래스로 train set 처리

### 전처리 방식
- 대화의 Question, Answer을 토큰화 후 ID로 변환(숫자 딕셔너리 데이터)
- 데이터 길이 표준화(padding)

### 함수 구조
- __getitem__의 인풋으로 dataframe의 row 번호를 넣으면 질문+답변, 마스크, 답변 r개요eturn
    - 단, 질문+답변: index 로 만든 질문+답변   <br/>
    -> token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
    - 단, mask = 질문길이 0 + 답변길이 1 + 나머지 0

In [7]:
idx = 0

In [8]:
data_tmp = train_set._data.iloc[idx]
print(data_tmp)
# Q: 질문
# A: 답변

Q            12시 땡!
A        하루가 또 가네요.
label             0
Name: 0, dtype: object


## 2.1. 전처리 프로세스

In [9]:
q = data_tmp["Q"]  # 질문을 가져온다.
q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.
print("1. 토큰화 전 raw data(대화의 Question):",q)

print("   > 참고. ChatbotDataset class에 선언된 q token:", train_set.q_token)
print("   > 참고. ChatbotDataset class에 선언된 sent token:", train_set.sent_token)
input_tmp = train_set.q_token + q + train_set.sent_token
print("2. tokenizer에 들어갈 인풋:",input_tmp)
q_toked = train_set.tokenizer.tokenize(input_tmp)
print("3. 토크나이징 결과:", q_toked)
print("4. 토큰을 ID로 변환:", train_set.tokenizer.convert_tokens_to_ids(q_toked))

1. 토큰화 전 raw data(대화의 Question): 12시 땡 
   > 참고. ChatbotDataset class에 선언된 q token: <usr>
   > 참고. ChatbotDataset class에 선언된 sent token: <unused1>
2. tokenizer에 들어갈 인풋: <usr>12시 땡 <unused1>
3. 토크나이징 결과: ['<usr>', '▁12', '시', '▁', '땡', '▁', '<unused1>']
4. 토큰을 ID로 변환: [2, 9349, 7888, 739, 7318, 739, 10]


## 2.2. 전처리 결과 해석
1. raw data -> ID
2. 문장 길이 통일을 위한 패딩

In [10]:
#질문+답변, 마스크, 답변
token_ids, mask, label = train_set.__getitem__(idx)
print(f"1. token_ids(질문+답변): {token_ids}") # 12시 땡! + 하루가 또 가네요.	
print(f"   > 참고. 문장 패딩 토큰 ID: {train_set.tokenizer.pad_token_id}")
print("    -> 위 리스트에서 3은 빈 칸을 패딩한 것")
print(f"   > 참고. 2 ~ 4까지가 질문에 해당하는 ID, 12557 ~ 1까지가 답변에 해당하는 ID")
print(f"2. mask(질문0+답변1+나머지단어0으로 패딩): {mask}")
print(f"   > 참고. Q 원 데이터: 학교종이 땡")
print(f"   > 참고. Q 토큰화 데이터: {q_toked}")
print("    -> 리스트 내 7개의 element를 [0 0 0 0 0 0 0] 마스크로 표현")
print("3. label:",label)

1. token_ids(질문+답변): [2, 9349, 7888, 739, 7318, 739, 10, 4, 12557, 6824, 9108, 9028, 7098, 8084, 739, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
   > 참고. 문장 패딩 토큰 ID: 3
    -> 위 리스트에서 3은 빈 칸을 패딩한 것
   > 참고. 2 ~ 4까지가 질문에 해당하는 ID, 12557 ~ 1까지가 답변에 해당하는 ID
2. mask(질문0+답변1+나머지단어0으로 패딩): [0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
   > 참고. Q 원 데이터: 학교종이 땡
   > 참고. Q 토큰화 데이터: ['<usr>', '▁12', '시', '▁', '땡', '▁', '<unused1>']
    -> 리스트 내 7개의 element를 [0 0 0 0 0 0 0] 마스크로 표현
3. label: [9, 9, 9, 9, 9, 9, 9, 12557, 6824, 9108, 9028, 7098, 8084, 739, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [11]:
print(f"- 문장 길이: {train_set.max_len}")
print(f"- token_ids 리스트 길이: {len(token_ids)}")

- 문장 길이: 40
- token_ids 리스트 길이: 40


# III. 모델 추가 학습

In [12]:
# base model
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [13]:
model.to(device)
model.train() # 추가 학습 준비

# 아래 프린트된 결과는 사전 학습 모델의 구조

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

- 학습 관련 파라미터 및 학습 방법(loss, optimization)

In [14]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18


### 참고. 파이토치에서는 backward() 함수, 즉 오차 역전파를 이용해 추가 학습 코드 만듦
- 아래는 PyTorch를 사용하여 기존 모델에 추가 학습을 적용하는 예시 코드([참고. pytorch 페이지](https://discuss.pytorch.kr/t/topic/2980/3))
    ~~~python

    import torch
    from transformers import GPT2Tokenizer, GPT2LMHeadModel

    # 모델과 토크나이저 로드
    model_name = 'xwin-mlewd-13b-v0.2.Q8_0.gguf'
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    # 새로운 데이터셋 준비
    new_data = ["새로운 소설 데이터 1", "새로운 소설 데이터 2", ...]

    # 데이터 전처리 및 토큰화
    encoded_new_data = [tokenizer.encode(data, add_special_tokens=True) for data in new_data]

    # DataLoader를 사용하여 학습 데이터 준비
    from torch.utils.data import DataLoader, TensorDataset

    input_ids = torch.tensor(encoded_new_data)
    dataset = TensorDataset(input_ids)
    data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # 추가 학습 시작
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

    for epoch in range(num_epochs):
        for batch in data_loader:
            inputs = batch[0].to(model.device)
            outputs = model(inputs, labels=inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(f"Epoch {epoch} Loss {loss.item()}")

    # 모델 저장
    model.save_pretrained('my_finetuned_model')
~~~

---------------------------
### 아래 loop를 통한 학습 과정을 line by line으로 확인

~~~python
print ("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # ---  평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # --- 학습 끝
        optimizer.step()
print ("end")
~~~

- 배치 데이터 샘플 확인

In [15]:
for batch_idx, samples in enumerate(train_dataloader):
    pass
print(batch_idx, samples)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/Users/yurikim/Desktop/study/koGPT2_ChatBot_ToyExample(local)/fine_tuning/train.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
/Users/yurikim/Desktop/study/koGPT2_ChatBot_ToyExample(local)/fine_tuning/train.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy

9 (tensor([[    2, 18351, 15413, 13340, 14807,    10,     4, 18351, 23638,  9337,
         11166, 12249,  8084,   739,     1,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2,  9244,  6958, 21246, 10811,    10,     4, 15365,  6824,  7801,
          8084,   739,     1,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 41998, 21716, 46339,  9456,  6853,  7991,    10,     4, 10556,
          9325, 46339,  9114,  6962, 13158,  8084,   739,     1,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2,  9226,  7889, 46651, 17003,  6

## 3.1. 추가 학습 데이터 중 일부(2개)만 가져와 사전학습 모델에 추가되는 과정 살펴보기

In [16]:
token_ids, mask, label = samples

- 추가 학습 데이터를 2개씩만 넣어서 추가 학습 프로세스 line by line으로 확인
    - 단, token_ids는 질문+답변+패딩으로 이루어진 토큰의 ID
- 참고. 사전학습 모델의 결과의 차원은 51,200
    - (lm_head): Linear(in_features=768, out_features=51200, bias=False)

### (1) Predict: token ID(전처리 결과)가 사전학습 모델을 거쳐 output shape이 51,200인 텐서 return

In [17]:
print(f"- token ID: {token_ids[0]}")
print(f"   > 참고. 인풋 토큰 아이디의 shape: {token_ids[0].shape}")
out = model(token_ids[:2]).logits
print(f"- 사전학습 모델의 seq2seq 결과(로지스틱 변환): {out}")
print(f"   > 참고. 사전학습 모델 결과의 shape: {out.shape}")

- token ID: tensor([    2, 18351, 15413, 13340, 14807,    10,     4, 18351, 23638,  9337,
        11166, 12249,  8084,   739,     1,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3])
   > 참고. 인풋 토큰 아이디의 shape: torch.Size([40])
- 사전학습 모델의 seq2seq 결과(로지스틱 변환): tensor([[[-5.6972, -5.7120, -4.1977,  ..., -1.4714, -2.5882, -1.3944],
         [-6.3349, -3.8235, -6.3412,  ..., -0.9051, -3.3118, -3.8610],
         [-4.1912, -5.0025, -5.3797,  ...,  0.7796, -2.1561, -2.0131],
         ...,
         [-1.3315,  0.3779, -0.7495,  ..., -0.4581,  4.5130, -1.9160],
         [-0.5138,  1.4653, -0.7271,  ...,  0.3596,  4.7405, -1.0524],
         [-0.7479,  0.3439, -0.2152,  ..., -0.5346,  3.6110, -0.2688]],

        [[-6.5312, -6.2856, -4.6215,  ..., -0.9085, -3.4982, -1.7145],
         [-3.7405, -5.1162, -3.7704,  ..., -5.7269, -3.5622, -3.1481],
         [-7.91

### (2.1) PREDICT 결과 포맷 조정: 
#### -> mask(질문 & 답변 길이 관련 벡터)의 차원을 token ID 결과와 맞춤 <br/> (단어가 있는 위치(질문0, 답변1) & 빈 칸(0)이 있는 위치 구분)
- 마지막 차원의 길이 51,200로 통일

In [18]:
print(f"- mask(질문0+답변1+나머지단어0으로 패딩): {mask[:2]}")
print(f"   > 참고. mask의 shape: {mask[:2].shape}")

mask_3d = mask[:2].unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
print(f"- mask_3d: {mask_3d}")
print(f"   > 참고. 사전학습 모델 결과의 shape: {mask_3d.shape}")
print(f"   > 예. 첫번째 데이터의 5번째 mask element(질문 토큰 위치): {mask_3d[0][4]}")
print(f"   > 예. 첫번째 데이터의 8번째 mask element(답변 토큰 위치): {mask_3d[0][12]}")

- mask(질문0+답변1+나머지단어0으로 패딩): tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
   > 참고. mask의 shape: torch.Size([2, 40])
- mask_3d: tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]])
   > 참고. 사전학습 모델 결과의 shape: torch.Size([2, 40, 51200])
   > 예. 첫번째 데이터의 5번째 mask element(질문 토큰 위치): tensor([0, 0, 0,  ..., 0, 0, 0])
   > 예. 첫번째 데이터의 8번째 mask element(답변 토큰 위치): tensor([1

### (2.2) PREDICT 결과 포맷 조정: 
#### -> 실제 단어가 있는 위치에만 모델 결과를 넣고 빈 칸은 0에 가까운 수 넣기
- mask 내 element 중
    - 실제 단어가 있는 위체 element에는 out(사전학습 모델 결과)를 반환하고, 
    - 빈칸을 padding으로 부분은 0에 가까운 아주 작은 수 Sneg(-1e+18)를 넣는다

In [19]:
#  torch.where: where(condition, input, other, *, out=None) -> Tensor
#       -> Return a tensor of elements selected from either :attr:`input` or :attr:`other`, depending on :attr:`condition`.
# --- 단어 element가 있는 부분에는 out(사전학습 모델 결과)를 반환하고, 빈칸이어서 padding인 부분은 0에 가까운 아주 작은 수 Sneg(-1e+18)를 넣는다
mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))

print(f"- 단어 위치(질문, 답변, 빈 칸) 고려 결과 조정: {mask_out}")
print(f"- 단어 위치 텐서의 shape: {mask_out.shape}")

- 단어 위치(질문, 답변, 빈 칸) 고려 결과 조정: tensor([[[-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         ...,
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18]],

        [[-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
        

In [20]:
print(f"   > 예. 첫번째 데이터의 5번째 mask element(질문 토큰 위치): {mask_out[0][4]}")
print(f"   > 예. 첫번째 데이터의 8번째 mask element(답변 토큰 위치): {mask_out[0][12]}")

   > 예. 첫번째 데이터의 5번째 mask element(질문 토큰 위치): tensor([-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
        -1.0000e+18, -1.0000e+18], grad_fn=<SelectBackward0>)
   > 예. 첫번째 데이터의 8번째 mask element(답변 토큰 위치): tensor([-4.4155, -4.1209, -4.4621,  ...,  0.0094, -4.1377, -4.1392],
       grad_fn=<SelectBackward0>)


### (3) LOSS 계산
#### -> Pedict 결과와 답변(answer) 간 차이

- 실제 답변(true y)

In [21]:
label[:2]

tensor([[    9,     9,     9,     9,     9,     9, 18351, 23638,  9337, 11166,
         12249,  8084,   739,     1,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    9,     9,     9,     9,     9,     9, 15365,  6824,  7801,  8084,
           739,     1,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3]])

In [22]:
label[:2].shape

torch.Size([2, 40])

- Question에 따른 예측값(predicted y)

In [23]:
mask_out.transpose(2, 1)

tensor([[[-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         ...,
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18]],

        [[-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         [-1.0000e+18, -1.0000e+18, -1.0000e+18,  ..., -1.0000e+18,
          -1.0000e+18, -1.0000e+18],
         ...,
         [-1.0000e+18, -1

In [24]:
mask_out.transpose(2, 1).shape

torch.Size([2, 51200, 40])

- 참고. criterion은 torch.nn.CrossEntropyLoss(reduction="none")으로 위에서 선언
    - 크로스엔트로피를 통해 predict 결과 평가

In [25]:
loss = criterion(mask_out.transpose(2, 1), label[:2])

In [26]:
loss

tensor([[10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435,  9.3041,  2.7851,
          6.4304,  4.0772,  5.9460,  8.1183,  4.1581, 17.1258, 14.9869, 10.8435,
         10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435,
         10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435,
         10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435],
        [10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8989,  7.3553,
          8.0691,  5.7411,  2.7540, 15.5434, 14.9512, 10.8435, 10.8435, 10.8435,
         10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435,
         10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435,
         10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435, 10.8435]],
       grad_fn=<ViewBackward0>)

### (4) loss를 역전파 학습

In [27]:
avg_loss = loss.sum() / mask[:2].sum()
avg_loss

tensor(52.0143, grad_fn=<DivBackward0>)

In [28]:
avg_loss.backward()

In [29]:
optimizer.step() # 학습 끝

In [30]:
# # 파일저장
 
# import pickle

model_pkl_file = "./checkpoints/GPT2LMHeadModel.pkl"  

# with open(model_pkl_file, 'wb') as file:  
#     pickle.dump(model, file)

- 위에서 학습 & 저장 파일 불러오기

In [31]:
import pickle

def load_model(filepath):
    with open(filepath, 'rb') as f:
        model = pickle.load(f)
    return model


# IV. 학습 모델(fine tuning 결과) 테스트 

In [32]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, 
            unk_token='<unk>', # out of vacabulary token
            pad_token=PAD, mask_token=MASK) 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


- 프롬프트를 인풋으로 받는 함수

In [33]:
?input

Signature: input(prompt='')
Docstring:
Forward raw_input to frontends

Raises
------
StdinNotImplementedError if active frontend doesn't support stdin.
File:      ~/anaconda3/envs/py_3_8/lib/python3.8/site-packages/ipykernel/kernelbase.py
Type:      method

In [34]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, 
            # unk_token='<unk>', # out of vacabulary token
            pad_token=PAD, mask_token=MASK) 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


#### 챗봇 실행

In [35]:
# 안녕 친구야 반가워"

In [36]:
# idx = 1
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1: # 문장 끝맺음(EOS)까지 무한 루프
            # input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0)
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT +  A_TKN + a)).unsqueeze(dim=0)
            pred = model(input_ids)
            pred = pred.logits
            
            # 생성된 값 중 가장 확률이 높은 단어 가져오기
            gen_list = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist()) # 결과 확인용
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]

            if gen == EOS:
                break
            
            # gen에서 고른 확률 높은 답변(단어) 추가
            a += gen.replace("▁", " ").replace('<unk>','')
            # if idx ==100:
            #     print("-----------------------")
            #     print(f"input: {Q_TKN + q + SENT + A_TKN + a}")
            #     # print(f"gen: {gen_list}")
            #     print(f"a: {a}")
            #     break
            # idx+=1
        print(f"input: {Q_TKN + q + SENT + A_TKN + a}")
        print("Chatbot > {}".format(a.strip()))


100